In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import combinations
from itertools import chain
from collections import namedtuple
import pickle
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
import os.path
import shutil
import inspect
import matplotlib.pyplot as plt
import json
from pprint import pprint
from copy import copy
from nn_lib import *
from train_utils import *
from cifar100_utils import *
from train_procedures import *
np.random.seed(1997)
ALGORITHM=sys.argv[1]
BASE_DIR=sys.argv[2]
if ALGORITHM=='full_data':
    CLASS_BATCH_SIZE=int(sys.argv[3])

In [3]:
data_dict=load_cifar100_data()

In [4]:
data_dict

{'X_test': array([[[[0.78039217, 0.84313726, 0.9764706 ],
          [0.76862746, 0.827451  , 0.95686275],
          [0.7647059 , 0.8235294 , 0.9529412 ],
          ...,
          [0.84705883, 0.90588236, 0.98039216],
          [0.8509804 , 0.90588236, 0.98039216],
          [0.8784314 , 0.91764706, 0.9882353 ]],
 
         [[0.77254903, 0.8235294 , 0.9372549 ],
          [0.7647059 , 0.8156863 , 0.93333334],
          [0.7647059 , 0.8235294 , 0.9411765 ],
          ...,
          [0.90588236, 0.9529412 , 0.98039216],
          [0.9137255 , 0.9529412 , 0.98039216],
          [0.94509804, 0.9607843 , 0.99215686]],
 
         [[0.87058824, 0.8862745 , 0.9647059 ],
          [0.8352941 , 0.8627451 , 0.9490196 ],
          [0.81960785, 0.85882354, 0.9529412 ],
          ...,
          [0.9529412 , 0.98039216, 0.9843137 ],
          [0.95686275, 0.9764706 , 0.9843137 ],
          [0.98039216, 0.98039216, 0.99215686]],
 
         ...,
 
         [[0.28235295, 0.28627452, 0.3882353 ],
        

In [4]:
# for testing
ALGORITHM="SupportNet"

In [5]:
#non-incremental settings
if ALGORITHM == 'ft':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':100,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#icarl settings
elif ALGORITHM == 'icarl':
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'icarl_exemplars',
                  'train_method':'train_distillation',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':100,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#SupportNet settings
elif ALGORITHM == 'SupportNet':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':100,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#ewc settings
elif ALGORITHM == 'ewc':
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':100,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
elif ALGORITHM == 'full_data':
    hyper_params={'beta':1e-5,
                  'initial_lr':2,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':100,
                  'base_dir':BASE_DIR,
                  'class_batch_size':CLASS_BATCH_SIZE,
                  'use_theoretical_mean':True}
else:
    assert False

In [6]:
tf_tensors,tf_variables,tf_networks=build_graph(hyper_params,fixed_params)

Use ResNet32


/home/liz0f/SupportNet/train_procedures.py:50: UserWarning: when use ewc, loss_function should be softmax
  warnings.warn('when use ewc, loss_function should be softmax')


Use ResNet32
Use ResNet32


In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
if ALGORITHM=='full_data':
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=1,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)
else:
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=50,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)

In [8]:
data_dict_iterations=data_dict_each_iteration("Aug31-cifar100-SupportNet-full2",50,2,data_dict)

In [9]:
eval_vals_lists_test=eval_vals_each_iteration("Aug31-cifar100-SupportNet-full2",50,2,hyper_params["test_batch_size"],tf_tensors,tf_networks,data_dict_iterations,"cifar100",sess)

/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0

In [13]:
with open("cifar100-output.csv",'w') as f:
    for i in range(0,len(eval_vals_lists_test)):
        for j in range(0,len(eval_vals_lists_test)):
            print(eval_vals_lists_test[i][j]["top1_accuracy"],end=",",file=f)
        print(file=f)